In [ ]:
import gc
import wandb
import pandas as pd
import torch
import torch.cuda

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.getcwd()

'/home/danbi/userdata/SGM_AI/darklady'

In [4]:
from pathlib import Path
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments

from transformers import TextDataset,DataCollatorForLanguageModeling

In [25]:
!pip install wandb
!pip install ipywidgets
!pip install scikit-learn
!pip install whatlies[all]

In [5]:
torch.__version__
torch.cuda.is_available()
!nvidia-smi

Mon Aug 22 15:11:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:02:00.0 Off |                  Off |
| 30%   34C    P8    15W / 230W |      8MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
def read_txt(file_path):
    with open(file_path, 'r', encoding ="UTF-8") as f:
    #with open(file_path, 'r') as f:
        fr = f.read()
    return fr

def read_csv(file_path):
    fr = pd.read_csv(file_path, encoding = "utf-8")
    return fr

In [7]:
dataset = read_csv('/home/danbi/userdata/SGM_AI/darklady/dataset/dataset_0815.csv')
dataset

,Unnamed: 0,동화
0,0,우리 친구들은 미술이 뭐라고 생각하나요. 종이에 그림을 그리는 것 아니면 멋진 ...
1,1,2022년 5월 5일은 방정환 선생님이 어린이날을 만든 지 100주년이 되는 날이랍...
2,2,5월은 가정의 달이라고 할 만큼 가족들의 소중함을 생각해보게 하는 기념일이 참 많은...
3,3,우리 친구들은 또래 친구들을 만나면 보통 어떤 놀이를 하면서 시간을 보내나요? ...
4,4,6월 6일은 나라를 지키기 위해 목숨을 바친 모든 이들의 넋을 기리는 법정 기념일 ...
...,...,...
1309,1309,옛날에 한 번은 마부가 포도주를 잔득 실은 ‘달구지’를 끌고 가다 그만 속도를 너무...
1310,1310,어느 커다란 도시에 나이 드신 어머님 한 분이 자신의 방에 저녁 내내 앉아 그동안 ...
1311,1311,가난한 시골 소년이 어느 날 교회에서 목사님이 “하늘나라 왕국에 들어가길 원하는 자...
1312,1312,어느 날 오후에 아기 예수가 아기침대에 누워 잠을 자고 있었답니다. 그때 그의 어머...


In [8]:
swapped_dataset = read_csv('/home/danbi/userdata/SGM_AI/darklady/dataset/swapped_data_0822.csv')
swapped_dataset#['동화']

,Unnamed: 0,동화
0,0,우리 친구들은 미술이 뭐라고 생각하나요. 종이에 그림을 그리는 것 아니면 멋진 ...
1,1,2022년 5월 5일은 <인물1> 선생님이 어린이날을 만든 지 100주년이 되는 날...
2,2,5월은 가정의 달이라고 할 만큼 가족들의 소중함을 생각해보게 하는 기념일이 참 많은...
3,3,우리 친구들은 또래 친구들을 만나면 보통 어떤 놀이를 하면서 시간을 보내나요? ...
4,4,6월 6일은 나라를 지키기 위해 목숨을 바친 모든 이들의 넋을 기리는 법정 기념일 ...
...,...,...
1309,1309,옛날에 한 번은 마부가 포도주를 잔득 실은 ‘달구지’를 끌고 가다 그만 속도를 너무...
1310,1310,어느 커다란 도시에 나이 드신 어머님 한 분이 자여신의 방에 저녁 내내 앉아 그동안...
1311,1311,가난한 시골 소녀가 어느 날 교회에서 목사님이 “하늘나라 왕국에 들어가길 원하는 자...
1312,1312,어느 날 오후에 아기 예수가 아기침대에 누워 잠을 자고 있었답니다. 그때 *그녀의 ...


In [9]:
concat_dataset = dataset + swapped_dataset
concat_dataset['동화'][0]

' 우리 친구들은 미술이 뭐라고 생각하나요.  종이에 그림을 그리는 것 아니면 멋진 조각 작품을 만드는 것 그런데 지금의 미술은 꼭 그림이나 조각품만이 아니랍니다.  비디오 아트 행위 예술 벽의 낙서까지 그 어떤 것도 예술이고 미술일 수 있다고 해요. 그럼 오늘 신나는 동화여행 속에서 알쏭달쏭 현대미술관으로 재미있고 엉뚱한 작품을 만나러 가볼까요.  알쏭달쏭 미술관 "나는 알쏭달쏭 미술관 관장 \'다다\'란다 이곳은 신기하고 괴상한 작품으로 가득하지 재미있는 미술 작품들이 가득한 알쏭달쏭 미술관에 온 걸 환영합니다." "이쪽에 있는 변기를 볼래? 화장실에서나 볼 수 있는 변기가 미술 작품이라면 믿겠니?" "네? 변기가 미술 작품이라구요?" "너희들 모나리자란 그림을 알고 있지? 레오나르도 다빈치가 그린 그림이야. 그런데 뒤샹이라는 화가는 모나리자의 얼굴에 수염만 그리고 전시를 했단다. 그래서 사람들은 깜짝 놀랐지." "네? 유명한 작품에 낙서를 한 게 미술 작품이라구요?" "뒤샹은 장난꾸러기였나봐. 그럼 이거는 뭘 그린 건지 한번 맞춰볼래? 여러 색깔의 물감을 뿌리고 붓고 흘린 거란다 곳곳에 커다란 물감 얼룩도 있지" "폴록이라는 화가는 이게 연보라빛 안개라는구나. 그런데 미술관 관장인 내가 봐도 신나게 물감을 뿌려 놓은 것 같거든 혹시 너희들 눈에는 안개가 보이니?" "글쎄요? 보이는 것도 같고 아닌 것 같기도 하고.." "그렇게 볼록은 커다란 종류의 물감을 쏟아 부었어. 그리고 붓을 휘젓고 물감을 튀겼지. 가끔은 물감 대신 모래를 톡톡 뿌리기도 했단다. 진짜 신났겠지?" "물감을 가지고 장난치는 것 같아..." "화가 에른스트는 물건의 종이를 대고 문지르면서 작품을 만들었어. 어떤 물건이냐에 따라 무늬가 달라지지!" "에이 이건 나도 그릴 수 있을 것 같아." "종이에 물감을 쭉 짜놓고 접었다가 펴면 양쪽에 똑같은 그림이 나타나지 이걸 \'데칼코마니\'라고 한단다." "이 그림은 어떠냐 이것도 미술 작품이야. 리엔텐 슈타인은 만화의 한 장면에 검은 선을 그

In [10]:
trainset, testset = train_test_split(concat_dataset['동화'], test_size=0.2, shuffle=True, random_state=123)
#trainset, testset

fw = open("/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_trainset.txt", "w", encoding = "utf-8")
fw.write(trainset.str.cat(sep=' ').replace('\n', ' '))
fw.close()

fw = open("/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_testset.txt", "w", encoding = "utf-8")
fw.write(testset.str.cat(sep=' ').replace('\n', ' '))
fw.close()

In [11]:
# from transformers import PreTrainedTokenizerFast
# from transformers import GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM 
import tqdm as notebook_tqdm

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2", use_cache = False,
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

In [12]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', use_cache = False).to(device='cuda', non_blocking=True)
torch.cuda.empty_cache()

In [13]:
prompt = """
옛날 옛적에 한 오누이가 살고 있었습니다.
"""
with torch.no_grad():
  tokens_bfr = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens_bfr = model.generate(tokens_bfr, do_sample=True, temperature=0.9, max_length=250, 
                              pad_token_id=tokenizer.pad_token_id,
                              eos_token_id=tokenizer.eos_token_id,
                              bos_token_id=tokenizer.bos_token_id)
  generated_bfr = tokenizer.batch_decode(gen_tokens_bfr)[0].replace("\n", " ")
  
print(generated_bfr)

 옛날 옛적에 한 오누이가 살고 있었습니다. 오누이는 뚱뚱하고 마른데다가 옷이 좀 무거웠습니다. 남편은 다 굶어 봤습니다. 그런데 오누이가 어떻게 되었는지 보려고 뚱뚱한 오누이를 보았습니다. 오누이도 키가 큰 데다가 늘씬한 체형의 몸이라 체중이 줄면 살이 찌기 쉽다고 들었습니다. 뚱뚱한 오누이를 보니 너무 큰 것이 아닐까 생각하였습니다. 오누이는 키가 크고 마른데다가 살이 찌는 것 같았습니다. 그래서 저는 몸을 늘여보고 다리를 늘여보고 다리를 늘여보고 다리를 늘여보고 다리를 늘여보는 등의 방법을 반복하여 보았습니다. 그러다가 보니 오누이는 키가 크고 마른데도 살이 찌었습니다. 이렇게 살이 찌는 것을 볼 수 있는 방법은 바로 다리를 늘여 보고 다리를 늘여보는 것이었습니다. 다리를 늘여보고 다리를 늘여보는 방법으로 다이어트에 성공한 것입니다. 저희 부부는 그 당시 저의 몸무게가 얼마나 줄었는지 모르지만 당시에는 제 모습이 아니었습니다. 살과 살의 관계를 보고 나니 가슴이 뭉클하였습니다. 이렇게 생각하니 몸도 커지고 몸이 작아졌다. 그러자 그제야 오누이는 조금 더 살이 쏙 빠지면서


In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_path = "/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_trainset.txt"
test_path = "/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_testset.txt"
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [39]:
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [123]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

training_args = TrainingArguments(
    output_dir="/home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0822", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=60, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    logging_steps=20,
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps= 15000, # after # steps model is saved 
    warmup_steps=400,# number of warmup steps for learning rate scheduler
    save_strategy = "steps",
    evaluation_strategy = "steps",
    prediction_loss_only=True,
    optim="adamw_torch",
    dataloader_pin_memory=False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

PyTorch: setting up devices


In [124]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#optimizer.zero_grad()
#with torch.no_grad():
torch.cuda.empty_cache()
trainer.train()

***** Running training *****
  Num examples = 16893
  Num Epochs = 60
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15840


Step,Training Loss,Validation Loss
100,0.210700,5.906689
200,0.230400,5.906446
300,0.243200,5.929402
400,0.281500,5.927177
500,0.305600,5.923293
600,0.277300,5.956725
700,0.296100,5.961024
800,0.284200,5.969594
900,0.274000,5.999148
1000,0.285700,5.999606


***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** 

In [113]:
trainer.save_model()

Saving model checkpoint to /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819
Configuration saved in /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/config.json
Model weights saved in /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/pytorch_model.bin


In [114]:
finetuned_model_path = "--"
finetuned_model = GPT2LMHeadModel.from_pretrained(finetuned_model_path).to(device='cuda', non_blocking=True)

loading configuration file /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_p

In [121]:
#existing code
prompt = """
옛날 옛적에 한 오누이가 살고 있었어요.
"""
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = finetuned_model.generate(tokens, 
                                        early_stopping=True,
                                        do_sample=True,
                                        temperature=0.8, 
                                        max_length=250, 
                                        repetition_penalty=3.0,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        unk_token_id=tokenizer.unk_token_id,)
  generated = tokenizer.decode(gen_tokens[0], skip_special_tokens=True).replace("\n", " ")
  
print(generated)

 옛날 옛적에 한 오누이가 살고 있었어요.  오빠는 너무도 가난하여 양식을 구할 길이 없어 아들아이를 가지길 간절히 바랬고, 자카따룹은 세상을 떠나고 말았어요. 그때 우연히 엄마 나나가 할머니에게 말하는 것을 얻었지요. “할머니, 저하고 결혼하세요.” 사랑하는 아빠를 섬기다 가버린 애가 하루는 꿈에 삼았던 그 새가 살아 있는 걸 보았답니다. 그녀가 창가로 달려가 정말 어느 새 큰 원 안에 갇혀 있게 되었는지 궁금해졌거든요.  그러자 왕비가 이야기를 시작했습니다. “이제 떠돌아다니는 삶에도 지쳤소. 그러니 내가 하느님께 나아갈 수 있도록 도와다오.” 공주가 대답하였어.  “아니,”라며 왕이 자기와 함께 동행한 아기 곰을 데려오셨는데, 그도 마찬가지로 구혼자들이 나타나 이렇게 말하며 그가 원하는 방향으로 헤엄쳐 갔다는 거예요, 그래서 그는 마침내 그를 엄마와 아이를 살려줄 용으로 변신시키고 왕비와 나누어 달라 청하리라 부르게 된 거지 뭐죠. 하지만 이 일로 왕비는 납북절한테는 어떤 약속 말고 그냥 묻어주시기만 하면 된다는 뜻이었습니다.  넌 분명 아름답고 영리한테서 그리고 아무 것도 얻을 수가 있는데 말이 있어 그리할 것이며, 그렇다면 그렇게 말하고 싶진 않으냐 그러면서
